In [1]:
from __future__ import division
from math import exp
import numpy as np
from numpy import *
from random import normalvariate
from datetime import datetime
import pandas as pd

In [2]:
trainData = '/Users/wangjian/Desktop/fm/test2/diabetes_train.txt'
testData = '/Users/wangjian/Desktop/fm/test2/diabetes_test.txt'
#先给出路径，后边进行open
#除了使用pd.read，还可以这样，先定义路径，然后使用open即可
featureNum=8

In [3]:
def normalize(l):
    s=[]
    for x in l:
        x=float(x-np.min(l))/(np.max(l)-np.min(l))
        s.append(x)
    return s

In [5]:
#定义分割函数，除了处理data集最后一行的数据集，还需要把除标签数据之外的normalize

def loadTrainDataSet(data):
    dataMat = []#储存特征数据
    labelMat = []#储存标签数据
    fr = open(data)
    #注意，这里fr也不是数据集
    for line in fr.readlines():
        currLine = line.strip().split(',')
        #每一行数据，以，间隔切分，相当于全部切开
        lineArr = []
        
        for i in xrange(featureNum):
            lineArr.append(float(currLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(currLine[-1]) * 2 - 1)
        #data文件中最后一列为标签0，1.通过*2-1之后可以划分为 +1，-1；

    scalar_dataMat = []
    for row in dataMat:
        scalar_row = normalize(row)
        scalar_dataMat.append(scalar_row)
    return scalar_dataMat, labelMat

In [6]:
#基本和上边相同
def loadTestDataSet(data):
    dataMat = []
    labelMat = []
    fr = open(data)
    for line in fr.readlines():
        currLine = line.strip().split(',')
        lineArr = []
        for i in xrange(featureNum):
            lineArr.append(float(currLine[i]))

        dataMat.append(lineArr)
        
        labelMat.append(float(currLine[-1]) * 2 - 1)
    scalar_dataMat = []
    for row in dataMat:
        scalar_row = normalize(row)
        scalar_dataMat.append(scalar_row)
    return scalar_dataMat, labelMat

In [7]:
#根据预测集映射分类结果
def sigmoid(x):
    return 1.0/(1+np.exp(-float(x)))

In [8]:
def stocGradAscent(dataMatrix, classLabels, k, iter):
    #dataMatrix用的是mat，classLabels列表
    m, n = shape(dataMatrix)
    alpha = 0.01
    #初始化
    #w = random.randn(n, 1)#n为特征的个数
    w = zeros((n, 1))
    w_0 = 0.
    v = normalvariate(0, 0.2) * ones((n, k))
    
    for it in xrange(iter):
        #print it，迭代的轮数
        for x in xrange(m):
            #注意，这里是用的是yi的转化公式，转化成平方项-交叉项
            inter_1 = dataMatrix[x] * v  
            inter_2 = multiply(dataMatrix[x], dataMatrix[x]) * multiply(v, v)
            interaction = sum(multiply(inter_1, inter_1) - inter_2) / 2.
            #参考P2的公式
            p = w_0 + dataMatrix[x] * w + interaction#yi的预测的输出
            #print "y: ",p 
            loss = sigmoid(classLabels[x] * p[0, 0]) - 1
            print "loss: ",loss

        
            #以下参数更新看不明白
            #当参数theta=w0时
            w_0 = w_0 - alpha * loss * classLabels[x]
            #对于w0为什么不是w_0 = w_0 - alpha * 1       
            
            for i in xrange(n):
                #参数theta=wj
                if dataMatrix[x, i] != 0:
                    w[i, 0] = w[i, 0] - alpha * loss * classLabels[x] * dataMatrix[x, i]
                    for j in xrange(k):
                        #参数theta=vj
                        v[i, j] = v[i, j] - alpha * loss * classLabels[x] * (dataMatrix[x, i] * inter_1[0, j] - v[i, j] * dataMatrix[x, i] * dataMatrix[x, i])
                        
    return w_0, w, v

In [9]:
def getAccuracy(dataMatrix, classLabels, w_0, w, v):
    m, n = shape(dataMatrix)
    #数据集前7列是特征，最后一列是分类，因此要利用前7列预测出类别去和test的第8列做比较
    #因此误差率=预测错误数／总数=预测错误的行／总的行
    allItem = 0
    error = 0
    result = []
    for x in xrange(m):
        allItem += 1
        inter_1 = dataMatrix[x] * v
        inter_2 = multiply(dataMatrix[x], dataMatrix[x]) * multiply(v, v)#multiply¶ÔÓ¦ÔªËØÏà³Ë
        #Íê³É½»²æÏî
        interaction = sum(multiply(inter_1, inter_1) - inter_2) / 2.
        p = w_0 + dataMatrix[x] * w + interaction#计算预测的输出
        
        
        pre = sigmoid(p[0, 0]) #根据预测数值指示分类的类别 
        '''result.append(pre)'''
        #计算错误率
        if pre < 0.5 and classLabels[x] == 1.0:
            error += 1
        elif pre >= 0.5 and classLabels[x] == -1.0:
            error += 1
        else:
            continue
        
    
    '''    print result'''
    
    return float(error) / allItem
        

In [10]:
if __name__ == '__main__':
    dataTrain, labelTrain = loadTrainDataSet(trainData)
    dataTest, labelTest = loadTestDataSet(testData)
    date_startTrain = datetime.now()
    print "开始训练"
    w_0, w, v = stocGradAscent(mat(dataTrain), labelTrain, 20, 5)
    print "训练准确性为：%f" % (1 - getAccuracy(mat(dataTrain), labelTrain, w_0, w, v))
    date_endTrain = datetime.now()
    print "训练时间：%s" % (date_endTrain - date_startTrain)
    print "开始测试"
    print "测试准确性为：%f" % (1 - getAccuracy(mat(dataTest), labelTest, w_0, w, v))  

开始训练
loss:  -0.4995611035984976
loss:  -0.5039754583520847
loss:  -0.499982472105738
loss:  -0.5034852256774737
loss:  -0.5010260938240978
loss:  -0.5023557888523904
loss:  -0.5000989904118815
loss:  -0.5024203113370669
loss:  -0.49878527238962056
loss:  -0.4986467445891841
loss:  -0.5048223500568887
loss:  -0.49876520865854335
loss:  -0.5043195927210837
loss:  -0.49625874386101465
loss:  -0.49307805079484723
loss:  -0.4946007840945358
loss:  -0.4880552636085903
loss:  -0.4894960914949158
loss:  -0.5191610819614834
loss:  -0.48458868490191676
loss:  -0.5171738373413424
loss:  -0.5119485281410268
loss:  -0.49247821066626774
loss:  -0.48912721229796985
loss:  -0.4805363028788272
loss:  -0.4768810736605559
loss:  -0.4805189025957004
loss:  -0.5266433354722315
loss:  -0.5250249201667834
loss:  -0.5172483290418823
loss:  -0.5142127822986013
loss:  -0.48658320721990334
loss:  -0.5163741209831441
loss:  -0.5088884367665985
loss:  -0.5056608577866938
loss:  -0.5081044724752433
loss:  -0.500020

loss:  -0.3576346859485404
loss:  -0.6075589857687075
loss:  -0.3958055453686613
loss:  -0.6212261886845114
loss:  -0.38603766874458045
loss:  -0.6028168544784693
loss:  -0.5717039342565067
loss:  -0.3690808186650204
loss:  -0.39823110579144483
loss:  -0.4126800854237258
loss:  -0.3820169710879623
loss:  -0.614677246931608
loss:  -0.5782180743870178
loss:  -0.37836305937090997
loss:  -0.6180215220148775
loss:  -0.3764171318236126
loss:  -0.6214341947499241
loss:  -0.3921452066803671
loss:  -0.37001515993738043
loss:  -0.3670002130924416
loss:  -0.38263530251239464
loss:  -0.6093641004410263
loss:  -0.5966374023073475
loss:  -0.4191668985368908
loss:  -0.6140937092064086
loss:  -0.3759101603750955
loss:  -0.5993471857389792
loss:  -0.41778140342269077
loss:  -0.36134314657820377
loss:  -0.35765874595920844
loss:  -0.3856335403513236
loss:  -0.3669662746276884
loss:  -0.5927710084021087
loss:  -0.37482374581386224
loss:  -0.3835605068489071
loss:  -0.3681766579830872
loss:  -0.3592298566

loss:  -0.6716644242536134
loss:  -0.6437683394630129
loss:  -0.5621026153378998
loss:  -0.6353060095722509
loss:  -0.4090523823790624
loss:  -0.40243277024285884
loss:  -0.36250852532388855
loss:  -0.3375648129556399
loss:  -0.34850898740821545
loss:  -0.6238951692086607
loss:  -0.4454284097086272
loss:  -0.35092320371370744
loss:  -0.6409780534507421
loss:  -0.5742514670178704
loss:  -0.33415659239522344
loss:  -0.36461122002178614
loss:  -0.3315132252335765
loss:  -0.627618844337573
loss:  -0.6207743002495758
loss:  -0.6288032951542684
loss:  -0.6258628691177219
loss:  -0.3640654034983202
loss:  -0.3570900957435489
loss:  -0.3543381326740941
loss:  -0.6169446530899418
loss:  -0.5883081509427279
loss:  -0.4028730401411056
loss:  -0.6214914676483936
loss:  -0.373810538105674
loss:  -0.4488031665405764
loss:  -0.4387406653058007
loss:  -0.3532526475410135
loss:  -0.36923210055081057
loss:  -0.36139871758832476
loss:  -0.34059370672272327
loss:  -0.33545437582186244
loss:  -0.5786405549

loss:  -0.3261061736067038
loss:  -0.6859420831796998
loss:  -0.3396223724097892
loss:  -0.310583336693134
loss:  -0.3172322292904637
loss:  -0.6692775762484754
loss:  -0.3292138180086833
loss:  -0.3261152282885422
loss:  -0.3621910168764382
loss:  -0.27393964812463056
loss:  -0.6693161032667838
loss:  -0.33568210737463533
loss:  -0.37635799505462575
loss:  -0.31283248766217053
loss:  -0.31830192499013166
loss:  -0.3201277096918509
loss:  -0.6398606112681078
loss:  -0.6574235096888434
loss:  -0.3532565184818144
loss:  -0.3168625589046419
loss:  -0.2916869096446725
loss:  -0.30319216726360443
loss:  -0.3515377259901624
loss:  -0.2991420160544407
loss:  -0.33065283398263656
loss:  -0.2943907588829875
loss:  -0.7211577010211302
loss:  -0.6663667586927843
loss:  -0.5861698287201387
loss:  -0.2913424322866107
loss:  -0.29811575386344125
loss:  -0.648859919898102
loss:  -0.6810842402317681
loss:  -0.6725571527281726
loss:  -0.3255638741870611
loss:  -0.32337179081866774
loss:  -0.31193740885

loss:  -0.3071682088275496
loss:  -0.6172381693755788
loss:  -0.40222941433763604
loss:  -0.6463218203114425
loss:  -0.3635163746528923
loss:  -0.6108582959809425
loss:  -0.5500852698039942
loss:  -0.32620293069232664
loss:  -0.3993695157808036
loss:  -0.4247382469768114
loss:  -0.36975709022101333
loss:  -0.6115664580279431
loss:  -0.5599545948122542
loss:  -0.3482021279808756
loss:  -0.6413353819159184
loss:  -0.3429228608942304
loss:  -0.6281393009571414
loss:  -0.3857222023915631
loss:  -0.3501961110380333
loss:  -0.34712346225857504
loss:  -0.36193936497770585
loss:  -0.6054708241367595
loss:  -0.5863167532621785
loss:  -0.4335428080559407
loss:  -0.610158123119775
loss:  -0.36097403427378616
loss:  -0.6008875212823697
loss:  -0.4307830527540861
loss:  -0.33554842128380347
loss:  -0.3268680878922391
loss:  -0.36935711601105425
loss:  -0.33564526060241406
loss:  -0.5797232501519078
loss:  -0.3504322200629606
loss:  -0.3693736390867558
loss:  -0.3406396995569524
loss:  -0.3273853937

loss:  -0.3419616482335468
loss:  -0.6378124721135654
loss:  -0.6141680992234324
loss:  -0.575606630173665
loss:  -0.622897446050796
loss:  -0.36830258981419095
loss:  -0.6958028724569933
loss:  -0.6517312722920352
loss:  -0.5325728033906074
loss:  -0.6422249848096815
loss:  -0.4325267259526495
loss:  -0.4185756850819252
loss:  -0.35715837663155015
loss:  -0.3160188726242449
loss:  -0.33362184031838693
loss:  -0.6192455692552852
loss:  -0.4765266418993984
loss:  -0.3412189738914978
loss:  -0.6476561345477228
loss:  -0.5515871435118739
loss:  -0.3093724100240989
loss:  -0.35971499278721697
loss:  -0.31082555984205473
loss:  -0.6246500619139945
loss:  -0.6086543099447215
loss:  -0.6318787295516717
loss:  -0.6292278188780205
loss:  -0.35117975226782727
loss:  -0.34160943410465616
loss:  -0.34582878521917226
loss:  -0.6117856425157098
loss:  -0.5681447374293356
loss:  -0.4165149659691184
loss:  -0.6222213277057226
loss:  -0.3683818149834306
loss:  -0.4774325990292071
loss:  -0.464299012622

loss:  -0.3187150270708248
loss:  -0.3728999301913438
loss:  -0.2528039084267154
loss:  -0.6712315285826771
loss:  -0.3371880043351828
loss:  -0.3933292102716941
loss:  -0.3021057627876371
loss:  -0.31891980333189895
loss:  -0.32163923203615996
loss:  -0.629751383251383
loss:  -0.6533990403547207
loss:  -0.3601754875726041
loss:  -0.3071408419962637
loss:  -0.2775886841307279
loss:  -0.29162277901346323
loss:  -0.3661473048160515
loss:  -0.27886340101532225
loss:  -0.34285682238352433
loss:  -0.2836600074336073
loss:  -0.736314998871936
loss:  -0.6555701105925489
loss:  -0.5593431318889112
loss:  -0.27646147070903315
loss:  -0.2848036673268176
loss:  -0.6314730079795564
loss:  -0.6867383674246897
loss:  -0.6761345389601088
loss:  -0.32043537577570536
loss:  -0.31655136685880925
loss:  -0.3048693712470435
loss:  -0.6643546704183093
loss:  -0.3180511337676314
loss:  -0.31215871305413934
loss:  -0.31551162878999295
loss:  -0.6866809923063906
loss:  -0.6428561245721507
loss:  -0.3475005498

loss:  -0.5546186845618486
loss:  -0.3436980100412941
loss:  -0.37088655003328275
loss:  -0.33086790842350056
loss:  -0.31444795766844136
loss:  -0.6469317579976734
loss:  -0.3155135684821575
loss:  -0.3634962181293302
loss:  -0.6617882391903416
loss:  -0.6424641534088618
loss:  -0.6321534241969798
loss:  -0.3428453911634567
loss:  -0.3078415548559229
loss:  -0.6574037071858018
loss:  -0.35605964111291455
loss:  -0.35534453077339834
loss:  -0.6300327485720818
loss:  -0.4278968164167162
loss:  -0.43791455459314543
loss:  -0.3634735344260247
loss:  -0.6148858588970121
loss:  -0.37910989050727295
loss:  -0.31521833390890586
loss:  -0.5954547864030975
loss:  -0.3437640827528399
loss:  -0.3455591986013884
loss:  -0.30539634866309595
loss:  -0.2690543592707242
loss:  -0.28512945693001523
loss:  -0.30819895858014823
loss:  -0.3714922484376664
loss:  -0.3178093684443253
loss:  -0.32139417806055803
loss:  -0.5708812611433498
loss:  -0.3491145461133046
loss:  -0.3321911508407708
loss:  -0.341602

In [ ]:
#整个fm模型的代码结构如上，分割数据集&初始化（normalize）--SGD训练参数集合（w0,w,V）-----结合参数预测¥分类-------计算错误率


In [ ]:
================================#与SVM模型做对比，提高程度不大=======================================

In [15]:
from sklearn import svm
clf = svm.SVC(C=0.8, kernel='rbf', gamma=20, decision_function_shape='ovr')
clf.fit(dataTrain, labelTrain)
y_hat = clf.predict(dataTest)
print "训练精度",clf.score(dataTrain, labelTrain) 

训练精度 0.738


In [12]:
def accuracy(test,labelTest):
    m=np.shape(labelTest)[0]
    error=0
    for x in range(m):
        if test[x] * labelTest[x] <0:
            error+=1
    accuracy=1-error/m
    return accuracy



In [16]:
print "预测精度:",accuracy(y_hat,labelTest)

预测精度: 0.664179104478
